This script evaluates the performance of different graph-aware architectures in a node classification problem. Several datasets are employed paying special attention to the homophily ratio.

In [4]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn


import utils
from gsp_utils.baselines_archs import GCNN_2L
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows

# SEED = 0
SEED = 15
PATH = 'results/diff_filters/'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:1


In [2]:
def print_full_results(accs, ellapsed_time, datasets, exps):
    mean_accs = accs.mean(axis=2)
    med_accs = np.median(accs, axis=2)
    std_accs = accs.std(axis=2)
    mean_t = ellapsed_time.mean(axis=2)

    for i, dataset_name in enumerate(datasets):
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])

        print(f'{dataset_name} (homophily ratio: {edge_hom:.3f})')
        for j, exp in enumerate(exps):
            print(f'\t- {exp["leg"]}:\tmean: {mean_accs[j,i]:.3f} - std: {std_accs[j,i]:.4f} - med: {med_accs[j,i]:.3f} - time: {mean_t[j,i]:.2f} mins')
        
        print()

def summary_table(accs, datasets, exps, median=False):
    mean_accs = accs.mean(axis=2)
    cols_name = []
    for dataset_name in datasets:
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])
        cols_name.append(f'{dataset_name} ({edge_hom:.2f})')

    index_name = [exp['leg'] for exp in exps]

    return DataFrame(mean_accs, columns=cols_name, index=index_name)


In [3]:
DATASETS = ['TexasDataset',  'WisconsinDataset', 'CornellDataset', 'CiteseerGraphDataset', 'CoraGraphDataset']

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': .01, 'norm': True, 'leg': 'GCN-normA'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': .01, 'norm': False, 'leg': 'GCN-A'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'GCN-normA-1'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'GCN-A-1'},
        {'model': 'GFGCN', 'L': 4, 'K': 2, 'h0': .01, 'norm': False, 'leg': 'GCN-A-4-2'},
        {'model': 'H-GFGCN', 'L': 2, 'K': 2, 'norm': True, 'leg': 'GCN-normH'},
        {'model': 'H-GFGCN', 'L': 2, 'K': 2, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN-A'},]

## Original Parameters

In [4]:
# Common parameters - training
N_RUNS = 25
N_EPOCHS = 200
LR = .01
WD = 5e-4

# Common parameters - GNN
N_LAYERS = 2
K = 2
# IN_DIM = feat.shape[1]
# OUT_DIM = n_class
HID_DIM = 16
DROPOUT = 0
ACT = nn.ReLU()
LAST_ACT = nn.LogSoftmax(dim=1)
LOSS_FN = nn.NLLLoss()

In [5]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):
    A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device)
    N = A.shape[0]
    in_dim = feat.shape[1]
    out_dim = n_class

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            best_accs[k,j,i] = np.max(acc["test"])
            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{best_accs[k,j,i]:.3f}', end=' - ')
        print()      
    print() 


print_full_results(best_accs, ellapsed_times, DATASETS, EXPS)
table = summary_table(best_accs, DATASETS, EXPS)

TexasDataset
0: 0.649 - 0.676 - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 - 0.730 - 0.108 - 0.541 - 0.622 - 0.595 - 0.595 - 0.108 - 0.811 - 
1: 0.649 - 0.757 - 0.108 - 0.757 - 0.108 - 0.568 - 0.703 - 0.649 - 0.595 - 0.108 - 0.865 - 
2: 0.676 - 0.703 - 0.108 - 0.649 - 0.108 - 0.595 - 0.676 - 0.568 - 0.541 - 0.108 - 0.784 - 
3: 0.676 - 0.730 - 0.108 - 0.703 - 0.108 - 0.595 - 0.622 - 0.649 - 0.568 - 0.108 - 0.757 - 
4: 0.649 - 0.730 - 0.108 - 0.757 - 0.108 - 0.595 - 0.649 - 0.649 - 0.486 - 0.108 - 0.757 - 
5: 0.676 - 0.757 - 0.108 - 0.703 - 0.108 - 0.514 - 0.649 - 0.541 - 0.541 - 0.108 - 0.811 - 
6: 0.649 - 0.730 - 0.108 - 0.649 - 0.108 - 0.676 - 0.676 - 0.649 - 0.622 - 0.108 - 0.811 - 
7: 0.676 - 0.649 - 0.108 - 0.595 - 0.108 - 0.459 - 0.649 - 0.649 - 0.432 - 0.108 - 0.865 - 
8: 0.649 - 0.730 - 0.108 - 0.703 - 0.108 - 0.486 - 0.568 - 0.649 - 0.622 - 0.108 - 0.838 - 
9: 0.676 - 0.703 - 0.108 - 0.757 - 0.108 - 0.459 - 0.757 - 0.568 - 0.486 - 0.108 - 0.784 - 
10: 0.703 - 0.730 - 0.108 - 0.649 - 0.108 - 0.514 - 0.649 - 0.676 - 0.514 - 0.108 - 0.757 - 
11: 0.

In [6]:
table

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.665946,0.592157,0.539459,0.71060,0.81392
Kipf-A,0.717838,0.578824,0.446486,0.65436,0.76372
GCN-normA,0.108108,0.019608,0.270270,0.69856,0.80584
GCN-A,0.696216,0.559216,0.574054,0.62272,0.72216
GCN-normA-1,0.108108,0.019608,0.270270,0.68836,0.77620
GCN-A-1,0.548108,0.477647,0.435676,0.63680,0.71756
GCN-A-4-2,0.654054,0.487843,0.580541,0.44868,0.56544
GCN-normH,0.610811,0.587451,0.499459,0.68404,0.77516
GCN-H,0.548108,0.476078,0.450811,0.63132,0.72000
W-GCN-normA,0.108108,0.019608,0.270270,0.69704,0.78272


## Other non-linearities

In [8]:
# Common parameters - training
N_RUNS = 25
N_EPOCHS = 200
LR = .01
WD = 5e-4

# Common parameters - GNN
N_LAYERS = 2
K = 2
HID_DIM = 16
DROPOUT = 0
ACT = nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()

In [9]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):
    A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device)
    N = A.shape[0]
    in_dim = feat.shape[1]
    out_dim = n_class

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            best_accs[k,j,i] = np.max(acc["test"])
            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{best_accs[k,j,i]:.3f}', end=' - ')
        print()      
    print()


print_full_results(best_accs, ellapsed_times, DATASETS, EXPS)
table2 = summary_table(best_accs, DATASETS, EXPS)

TexasDataset
0: 0.649 - 0.730 - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 - 0.622 - 0.108 - 0.676 - 0.703 - 0.757 - 0.622 - 0.108 - 0.757 - 
1: 0.676 - 0.730 - 0.108 - 0.703 - 0.108 - 0.676 - 0.676 - 0.703 - 0.703 - 0.108 - 0.784 - 
2: 0.703 - 0.162 - 0.108 - 0.649 - 0.108 - 0.676 - 0.703 - 0.676 - 0.703 - 0.108 - 0.811 - 
3: 0.676 - 0.649 - 0.108 - 0.730 - 0.108 - 0.676 - 0.676 - 0.757 - 0.649 - 0.108 - 0.784 - 
4: 0.649 - 0.703 - 0.108 - 0.757 - 0.108 - 0.649 - 0.676 - 0.784 - 0.649 - 0.108 - 0.784 - 
5: 0.703 - 0.730 - 0.108 - 0.676 - 0.108 - 0.676 - 0.676 - 0.784 - 0.649 - 0.108 - 0.838 - 
6: 0.703 - 0.162 - 0.108 - 0.784 - 0.108 - 0.703 - 0.676 - 0.703 - 0.622 - 0.108 - 0.838 - 
7: 0.703 - 0.676 - 0.108 - 0.811 - 0.108 - 0.676 - 0.676 - 0.730 - 0.649 - 0.108 - 0.838 - 
8: 0.676 - 0.730 - 0.108 - 0.649 - 0.108 - 0.676 - 0.676 - 0.757 - 0.676 - 0.108 - 0.865 - 
9: 0.676 - 0.135 - 0.108 - 0.622 - 0.108 - 0.622 - 0.649 - 0.784 - 0.622 - 0.108 - 0.757 - 
10: 0.676 - 0.649 - 0.108 - 0.541 - 0.108 - 0.676 - 0.676 - 0.703 - 0.703 - 0.108 - 0.838 - 
11: 0.

In [10]:
table2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.676757,0.632157,0.591351,0.57768,0.55828
Kipf-A,0.605405,0.466667,0.441081,0.66864,0.77220
GCN-normA,0.108108,0.019608,0.270270,0.70652,0.80508
GCN-A,0.675676,0.593725,0.512432,0.64432,0.75012
GCN-normA-1,0.108108,0.019608,0.270270,0.69264,0.78152
GCN-A-1,0.677838,0.598431,0.492973,0.65772,0.74848
GCN-A-4-2,0.669189,0.516863,0.508108,0.54160,0.46380
GCN-normH,0.738378,0.773333,0.619459,0.42520,0.46624
GCN-H,0.680000,0.618824,0.501622,0.65624,0.74808
W-GCN-normA,0.108108,0.019608,0.270270,0.70028,0.77696


## Best Params GF-GNN

In [4]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 25
N_EPOCHS = 200  # 500
LR = .05
WD = .01
DROPOUT = 0

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 50

## Model params
h0 = .1  # 1
ACT = nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [5]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):
    A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device)
    N = A.shape[0]
    in_dim = feat.shape[1]
    out_dim = n_class

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            best_accs[k,j,i] = np.max(acc["test"])
            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{best_accs[k,j,i]:.3f}', end=' - ')
        print()      
    print()


print_full_results(best_accs, ellapsed_times, DATASETS, EXPS)
table3 = summary_table(best_accs, DATASETS, EXPS)

TexasDataset
0: 0.703 - 0.703 - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 - 0.892 - 0.865 - 0.892 - 0.108 - 0.784 - 
1: 0.703 - 0.703 - 0.108 - 0.865 - 0.703 - 0.865 - 0.108 - 0.757 - 
2: 0.703 - 0.649 - 0.108 - 0.946 - 0.784 - 0.919 - 0.108 - 0.757 - 
3: 0.703 - 0.703 - 0.108 - 0.865 - 0.703 - 0.865 - 0.108 - 0.811 - 
4: 0.730 - 0.703 - 0.108 - 0.892 - 0.865 - 0.892 - 0.108 - 0.784 - 
5: 0.703 - 0.730 - 0.108 - 0.892 - 0.865 - 0.892 - 0.108 - 0.730 - 
6: 0.730 - 0.703 - 0.108 - 0.892 - 0.811 - 0.919 - 0.108 - 0.730 - 
7: 0.703 - 0.730 - 0.108 - 0.892 - 0.865 - 0.865 - 0.108 - 0.730 - 
8: 0.730 - 0.649 - 0.108 - 0.865 - 0.838 - 0.892 - 0.108 - 0.757 - 
9: 0.703 - 0.649 - 0.108 - 0.892 - 0.784 - 0.892 - 0.108 - 0.784 - 
10: 0.703 - 0.676 - 0.108 - 0.649 - 0.784 - 0.865 - 0.108 - 0.757 - 
11: 0.703 - 0.676 - 0.108 - 0.892 - 0.730 - 0.892 - 0.108 - 0.757 - 
12: 0.730 - 0.703 - 0.108 - 0.865 - 0.784 - 0.919 - 0.108 - 0.757 - 
13: 0.703 - 0.676 - 0.108 - 0.865 - 0.892 - 0.865 - 0.108 - 0.757 - 
14: 0.703 - 0.730 - 0.108 - 0.892 - 0.757 - 0.892 - 0.108 - 0.7

In [ ]:
table3

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.708108,0.669804,0.622703,0.23964,0.31980
Kipf-A,0.678919,0.548235,0.445405,0.58728,0.76880
A-GCN-normA,0.108108,0.019608,0.270270,0.72372,0.81600
A-GCN,0.880000,0.868235,0.833514,0.54984,0.57056
H-GCN-normH,0.792432,0.721569,0.708108,0.23836,0.32280
GCN-H,0.888649,0.858824,0.867027,0.53104,0.55716
W-GCN-normA,0.108108,0.019608,0.270270,0.24792,0.31248
W-GCN,0.756757,0.701961,0.615135,0.50908,0.42452


In [7]:
table3  # h0 = .1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.705946,0.670588,0.622703,0.23912,0.31980
Kipf-A,0.678919,0.543529,0.445405,0.58748,0.77108
A-GCN-normA,0.108108,0.019608,0.270270,0.64768,0.80204
A-GCN,0.897297,0.841569,0.873514,0.54676,0.57720
H-GCN-normH,0.792432,0.721569,0.708108,0.23836,0.32280
GCN-H,0.888649,0.858824,0.867027,0.53104,0.55716
W-GCN-normA,0.108108,0.019608,0.270270,0.24792,0.31248
W-GCN,0.756757,0.701961,0.615135,0.50908,0.42452
